# How to connect to mysql using python and import the csv file into mysql  from-Jupyter-Notebook

In ordering to keep safe our credentials we save our credentials in a file **credentials.json**

```
{
  "host":"sql11.freesqldatabase.com",
  "database":"xxxxxx",
  "user":"xxxxxxx",
  "password":"xxxxxx"

}
```

In [224]:
# Python program to read
# json file
import json
# Opening JSON file
f = open('credentials.json')

In [225]:
# returns JSON object as
# a dictionary
data = json.load(f)

In [226]:
# You can uncommnet to see if you put correct credentials
#for i in data:
#    print(data[i])

In [227]:
#You can identify the database you have created
data['host']

'sql11.freesqldatabase.com'

In [228]:
import mysql.connector

In [229]:
import mysql.connector
from mysql.connector import Error

# Checking our Mysql connection

In [44]:
connection = mysql.connector.connect(host=data['host'],
                                         database=data['database'],
                                         user=data['user'],
                                         password=data['password'])

In [45]:
try:
    connection = mysql.connector.connect(host=data['host'],
                                             database=data['database'],
                                             user=data['user'],
                                             password=data['password'])
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  5.5.62-0ubuntu0.14.04.1
You're connected to database:  ('sql11501562',)
MySQL connection is closed


# Create MySQL table from Python 

Create MySQL table from Python
Now you know how to connect to a MySQL server from Python, In this section, we will learn how to create a table in MySQL from Python. Let’s create table ‘Laptop’ under the ‘Electronics’ database.

In [47]:
import mysql.connector

try:
    connection = mysql.connector.connect(host=data['host'],
                                             database=data['database'],
                                             user=data['user'],
                                             password=data['password'])

    mySql_Create_Table_Query = """CREATE TABLE Laptop ( 
                             Id int(11) NOT NULL,
                             Name varchar(250) NOT NULL,
                             Price float NOT NULL,
                             Purchase_date Date NOT NULL,
                             PRIMARY KEY (Id)) """

    cursor = connection.cursor()
    result = cursor.execute(mySql_Create_Table_Query)
    print("Laptop Table created successfully ")

except mysql.connector.Error as error:
    print("Failed to create table in MySQL: {}".format(error))
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Failed to create table in MySQL: 1050 (42S01): Table 'Laptop' already exists
MySQL connection is closed


# Create MySQL table with Pandas

In [168]:
import pandas as pd

## Create an in-memory SQLite database.

In [53]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [54]:
df = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df

,name
0,User 1
1,User 2
2,User 3


In [55]:
df.to_sql('users', con=engine)

An sqlalchemy.engine.Connection can also be passed to con:

In [56]:
with engine.begin() as connection:
    df1 = pd.DataFrame({'name' : ['User 4', 'User 5']})
    df1.to_sql('users', con=connection, if_exists='append')

This is allowed to support operations that require that the same DBAPI connection is used for the entire operation.

In [57]:
df2 = pd.DataFrame({'name' : ['User 6', 'User 7']})
df2.to_sql('users', con=engine, if_exists='append')

In [58]:
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'),
 (1, 'User 2'),
 (2, 'User 3'),
 (0, 'User 4'),
 (1, 'User 5'),
 (0, 'User 6'),
 (1, 'User 7')]

## Create an MySQL database with Pandas in real Database

## First method with mysql.connector 

### Step 1: Prepare the CSV File
To begin, prepare the CSV file that you'd like to import to MySQL. For example, I prepared a simple CSV file with the following data:

In [90]:
df=pd.read_csv('gpu.csv')  

In [91]:
df.head()

,Product Name,GPU Chip,Released,Bus,Memory,GPU clock,Memory clock
0,GeForce RTX 3060,GA106,"Jan 12th, 2021",PCIe 4.0 x16,"12 GB, GDDR6, 192 bit",1320 MHz,1875 MHz
1,GeForce RTX 3060 Ti,GA104,"Dec 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1410 MHz,1750 MHz
2,GeForce RTX 3070,GA104,"Sep 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1500 MHz,1750 MHz
3,GeForce RTX 3080,GA102,"Sep 1st, 2020",PCIe 4.0 x16,"10 GB, GDDR6X, 320 bit",1440 MHz,1188 MHz
4,Radeon RX 6600 XT,Navi 23,"Jul 30th, 2021",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1968 MHz,2000 MHz


In [92]:
df.columns

Index(['Product Name', 'GPU Chip', 'Released', 'Bus', 'Memory', 'GPU clock',
       'Memory clock'],
      dtype='object')

In [93]:
import pandas
from pandas.io.json import build_table_schema

In [94]:
build_table_schema(df)

{'fields': [{'name': 'index', 'type': 'integer'},
  {'name': 'Product Name', 'type': 'string'},
  {'name': 'GPU Chip', 'type': 'string'},
  {'name': 'Released', 'type': 'string'},
  {'name': 'Bus', 'type': 'string'},
  {'name': 'Memory', 'type': 'string'},
  {'name': 'GPU clock', 'type': 'string'},
  {'name': 'Memory clock', 'type': 'string'}],
 'primaryKey': ['index'],
 'pandas_version': '0.20.0'}

In [95]:
 data_set=df

In [96]:
num_cols = len(data_set.axes[1])

In [97]:
print(num_cols)

7


In [98]:
col_count = [len(l.split(",")) for l in data_set.columns]

In [99]:
print(type(col_count)) 

<class 'list'>


In [100]:
 print(len(col_count))

7


First, I establish the CREATE TABLE command and table name – ‘gpu_data’ – for this example, storing it in an ‘SQL_CREATE_TBL’ variable:

In [143]:
SQL_CREATE_TBL = "CREATE TABLE gpu_data("

In [146]:
for name in range(0, len(col_count)):
    column_name=data_set.columns[name].replace(' ', '_')
    SQL_CREATE_TBL += "{} TEXT, ".format(column_name)

Let’s view the ‘SQL_CREATE_TBL’ string variable contents once the for loop completes:

In [145]:
SQL_CREATE_TBL 

'CREATE TABLE gpu_data(Product_Name TEXT, GPU_Chip TEXT, Released TEXT, Bus TEXT, Memory TEXT, GPU_clock TEXT, Memory_clock TEXT, '

In [104]:
SQL_CREATE_TBL = SQL_CREATE_TBL.rstrip(" ,")

All that’s left to complete the CREATE TABLE statement, is simply append the closing parenthesis and semicolon to the ‘SQL_CREATE_TBL’ variable:

In [105]:
SQL_CREATE_TBL += ");"

In [106]:
SQL_CREATE_TBL 

'CREATE TABLE so_data(Product Name TEXT, GPU Chip TEXT, Released TEXT, Bus TEXT, Memory TEXT, GPU clock TEXT, Memory clock TEXT);'

The entire code structure for the dynamic CREATE TABLE statement is shown below:

In [151]:
SQL_CREATE_TBL = "CREATE TABLE gpu_data("
for name in range(0, len(col_count)):
     column_name=data_set.columns[name].replace(' ', '_')
     SQL_CREATE_TBL += "{} TEXT, ".format(column_name)
SQL_CREATE_TBL = SQL_CREATE_TBL.rstrip(" ,")
SQL_CREATE_TBL += ");"

In [152]:
SQL_CREATE_TBL

'CREATE TABLE gpu_data(Product_Name TEXT, GPU_Chip TEXT, Released TEXT, Bus TEXT, Memory TEXT, GPU_clock TEXT, Memory_clock TEXT);'

In [153]:
SQL_CREATE_TBL = "CREATE TABLE gpu_data("
for name in range(0, len(col_count)):
        column_name=data_set.columns[name].replace(' ', '_')
        SQL_CREATE_TBL += "{} varchar(255), ".format(column_name)
SQL_CREATE_TBL = SQL_CREATE_TBL.rstrip(" ,")
SQL_CREATE_TBL += ")"

In [155]:
SQL_CREATE_TBL

'CREATE TABLE gpu_data(Product_Name varchar(255), GPU_Chip varchar(255), Released varchar(255), Bus varchar(255), Memory varchar(255), GPU_clock varchar(255), Memory_clock varchar(255))'

## Step 2: Connect to the MySQL using Python and create a Database

Create a connection object to connect to MySQL, The connect() constructor creates a connection to the MySQL and returns a MySQLConnection object.

In [117]:
connection = mysql.connector.connect(    host=data['host'],
                                         database=data['database'],
                                         user=data['user'],
                                         password=data['password'])

## Step 3: Create a table and Import the CSV data into the MySQL table
We will create an gpus table under the our  database and insert the records in MySQL with below python code.

In [156]:
cursor.close()
connection.close()

In [157]:
import mysql.connector as msql
from mysql.connector import Error
try:
    #conn = mysql.connect(host='localhost', database='employee', user='root', password='root@123')
    
    conn = mysql.connector.connect(host=data['host'],database=data['database'],user=data['user'],password=data['password'])
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS gpu_data;')
        print('Creating table....')
# in the below line please pass the create table statement which you want #to create
        cursor.execute(SQL_CREATE_TBL)
        print("Table is created....")
        #loop through the data frame
        for i,row in df.iterrows():
            #here %S means string values 
            sql = "INSERT INTO "+data['database']+".gpu_data VALUES (%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            conn.commit()
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('sql11501562',)
Creating table....
Table is created....
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


## Step 5 : Query the Table
Query the table to make sure that our inserted data has been saved correctly.

In [158]:
# Execute query
sql = "SELECT * FROM "+data['database']+".gpu_data"
#sql = "SELECT * FROM employee.employee_data"

In [159]:
cursor.execute(sql)

In [160]:
# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

('GeForce RTX 3060', 'GA106', 'Jan 12th, 2021', 'PCIe 4.0 x16', '12 GB, GDDR6, 192 bit', '1320 MHz', '1875 MHz')
('GeForce RTX 3060 Ti', 'GA104', 'Dec 1st, 2020', 'PCIe 4.0 x16', '8 GB, GDDR6, 256 bit', '1410 MHz', '1750 MHz')
('GeForce RTX 3070', 'GA104', 'Sep 1st, 2020', 'PCIe 4.0 x16', '8 GB, GDDR6, 256 bit', '1500 MHz', '1750 MHz')
('GeForce RTX 3080', 'GA102', 'Sep 1st, 2020', 'PCIe 4.0 x16', '10 GB, GDDR6X, 320 bit', '1440 MHz', '1188 MHz')
('Radeon RX 6600 XT', 'Navi 23', 'Jul 30th, 2021', 'PCIe 4.0 x8', '8 GB, GDDR6, 128 bit', '1968 MHz', '2000 MHz')
('GeForce RTX 2060', 'TU106', 'Jan 7th, 2019', 'PCIe 3.0 x16', '6 GB, GDDR6, 192 bit', '1365 MHz', '1750 MHz')
('Radeon RX 6600', 'Navi 23', 'Oct 13th, 2021', 'PCIe 4.0 x8', '8 GB, GDDR6, 128 bit', '1626 MHz', '1750 MHz')
('GeForce RTX 3050 8 GB', 'GA106', 'Jan 4th, 2022', 'PCIe 4.0 x8', '8 GB, GDDR6, 128 bit', '1552 MHz', '1750 MHz')
('GeForce RTX 3090 Ti', 'GA102', 'Jan 27th, 2022', 'PCIe 4.0 x16', '24 GB, GDDR6X, 384 bit', '1560

# Using Pandas SQL Query Method

In [163]:
import mysql.connector

In [164]:
connection = mysql.connector.connect(host=data['host'],
                                         database=data['database'],
                                         user=data['user'],
                                         password=data['password'])

## Step 6: Run Pandas SQL Query method

In [165]:
df_remote = pd.read_sql_query(sql, con = connection)
df_remote

,Product_Name,GPU_Chip,Released,Bus,Memory,GPU_clock,Memory_clock
0,GeForce RTX 3060,GA106,"Jan 12th, 2021",PCIe 4.0 x16,"12 GB, GDDR6, 192 bit",1320 MHz,1875 MHz
1,GeForce RTX 3060 Ti,GA104,"Dec 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1410 MHz,1750 MHz
2,GeForce RTX 3070,GA104,"Sep 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1500 MHz,1750 MHz
3,GeForce RTX 3080,GA102,"Sep 1st, 2020",PCIe 4.0 x16,"10 GB, GDDR6X, 320 bit",1440 MHz,1188 MHz
4,Radeon RX 6600 XT,Navi 23,"Jul 30th, 2021",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1968 MHz,2000 MHz
5,GeForce RTX 2060,TU106,"Jan 7th, 2019",PCIe 3.0 x16,"6 GB, GDDR6, 192 bit",1365 MHz,1750 MHz
6,Radeon RX 6600,Navi 23,"Oct 13th, 2021",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1626 MHz,1750 MHz
7,GeForce RTX 3050 8 GB,GA106,"Jan 4th, 2022",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1552 MHz,1750 MHz
8,GeForce RTX 3090 Ti,GA102,"Jan 27th, 2022",PCIe 4.0 x16,"24 GB, GDDR6X, 384 bit",1560 MHz,1313 MHz
9,GeForce RTX 3070 Ti,GA104,"May 31st, 2021",PCIe 4.0 x16,"8 GB, GDDR6X, 256 bit",1575 MHz,1188 MHz


In [166]:
cursor.close()
connection.close()

## Second method with sqlalchemy 

In [ ]:
df=pd.read_csv('gpu.csv')  

In [187]:
import pandas as pd
from sqlalchemy import create_engine


In [191]:
infile = 'gpu.csv'
db = data['database']
db_tbl_name = 'gpu_data2'

In [210]:
'''
Load a csv file into a dataframe; if csv does not have headers, use the headers arg to create a list of headers; rename unnamed columns to conform to mysql column requirements
'''
def csv_to_df(infile, headers = []):
    if len(headers) == 0:
        df = pd.read_csv(infile)
   
    else:
        df = pd.read_csv(infile, header = None)
        df.columns = headers
 
    for r in range(10):
        try:
            df.rename( columns={'Unnamed: {0}'.format(r):'Unnamed{0}'.format(r)},    inplace=True )
            
        except:
            pass
    df.columns = df.columns.str.replace(' ', '_')
    return df


In [198]:
'''
Create a mapping of df dtypes to mysql data types local
'''
def dtype_mapping_local():
    return {'object' : 'TEXT',
        'int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}

In [199]:
'''
Create a mapping of df dtypes to mysql data types remote (not perfect, but close enough)
'''
def dtype_mapping():
    return {'object' : 'VARCHAR(255)',
        'int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'VARCHAR(255)',
        'timedelta[ns]' : 'VARCHAR(255)'}

In [200]:
'''
Create a sqlalchemy engine local
'''
def mysql_engine_local(user = 'root', password = 'abc', host = '127.0.0.1', port = '3306', database = 'a001_db'):
    engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format(user, password, host, port, database))
    return engine

In [201]:
'''
Create a sqlalchemy engine remote
'''
def mysql_engine_remote(user = data['user'], 
                        password = data['password'], 
                        host = data['host'], 
                        port = '3306', 
                        database = data['database']):
    engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format(user, password, host, port, database))
    return engine

In [203]:
'''
Create a mysql connection from sqlalchemy engine
'''
def mysql_conn(engine):
    conn = engine.raw_connection()
    return conn

In [204]:
'''
Create sql input for table names and types
'''
def gen_tbl_cols_sql(df):
    dmap = dtype_mapping()
    sql = "pi_db_uid INT AUTO_INCREMENT PRIMARY KEY"
    df1 = df.rename(columns = {"" : "nocolname"})
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for hl in hdrs_list:
        sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql

In [205]:
'''
Create a mysql table from a df
'''
def create_mysql_tbl_schema(df, conn, db, tbl_name):
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "USE {0}; CREATE TABLE {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()

In [206]:
'''
Write df data to newly create mysql table
'''
def df_to_mysql(df, engine, tbl_name):
    df.to_sql(tbl_name, engine, if_exists='replace')

In [211]:
df = csv_to_df(infile)
df.head()

,Product_Name,GPU_Chip,Released,Bus,Memory,GPU_clock,Memory_clock
0,GeForce RTX 3060,GA106,"Jan 12th, 2021",PCIe 4.0 x16,"12 GB, GDDR6, 192 bit",1320 MHz,1875 MHz
1,GeForce RTX 3060 Ti,GA104,"Dec 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1410 MHz,1750 MHz
2,GeForce RTX 3070,GA104,"Sep 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1500 MHz,1750 MHz
3,GeForce RTX 3080,GA102,"Sep 1st, 2020",PCIe 4.0 x16,"10 GB, GDDR6X, 320 bit",1440 MHz,1188 MHz
4,Radeon RX 6600 XT,Navi 23,"Jul 30th, 2021",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1968 MHz,2000 MHz


In [212]:
create_mysql_tbl_schema(df, mysql_conn(mysql_engine_remote()), db, db_tbl_name)

In [213]:
df_to_mysql(df, mysql_engine_remote(), db_tbl_name)

In [220]:
connection = mysql.connector.connect(    host=data['host'],
                                         database=data['database'],
                                         user=data['user'],
                                         password=data['password'])

In [221]:
# Execute query
sql = "SELECT * FROM "+data['database']+".gpu_data2"


In [222]:
df_remote_2 = pd.read_sql_query(sql, con = connection)
df_remote_2.head(10)

,index,Product_Name,GPU_Chip,Released,Bus,Memory,GPU_clock,Memory_clock
0,0,GeForce RTX 3060,GA106,"Jan 12th, 2021",PCIe 4.0 x16,"12 GB, GDDR6, 192 bit",1320 MHz,1875 MHz
1,1,GeForce RTX 3060 Ti,GA104,"Dec 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1410 MHz,1750 MHz
2,2,GeForce RTX 3070,GA104,"Sep 1st, 2020",PCIe 4.0 x16,"8 GB, GDDR6, 256 bit",1500 MHz,1750 MHz
3,3,GeForce RTX 3080,GA102,"Sep 1st, 2020",PCIe 4.0 x16,"10 GB, GDDR6X, 320 bit",1440 MHz,1188 MHz
4,4,Radeon RX 6600 XT,Navi 23,"Jul 30th, 2021",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1968 MHz,2000 MHz
5,5,GeForce RTX 2060,TU106,"Jan 7th, 2019",PCIe 3.0 x16,"6 GB, GDDR6, 192 bit",1365 MHz,1750 MHz
6,6,Radeon RX 6600,Navi 23,"Oct 13th, 2021",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1626 MHz,1750 MHz
7,7,GeForce RTX 3050 8 GB,GA106,"Jan 4th, 2022",PCIe 4.0 x8,"8 GB, GDDR6, 128 bit",1552 MHz,1750 MHz
8,8,GeForce RTX 3090 Ti,GA102,"Jan 27th, 2022",PCIe 4.0 x16,"24 GB, GDDR6X, 384 bit",1560 MHz,1313 MHz
9,9,GeForce RTX 3070 Ti,GA104,"May 31st, 2021",PCIe 4.0 x16,"8 GB, GDDR6X, 256 bit",1575 MHz,1188 MHz


In [223]:
connection.close()

In [ ]:
# Closing json file
f.close()